In [2]:
%run final_project_NLP_2.ipynb

['OTHER', 'OTHER', 'OTHER', 'OTHER', 'ORG', 'ORG', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'ORG', 'ORG', 'ORG', 'ORG', 'ORG', 'ORG', 'ORG', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'DATE', 'DATE', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'CARDINAL', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'ORG', 'OTHER', 'CARDINAL', 'CARDINAL', 'CARDINAL', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'NORP', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'CARDINAL', 'OTHER', 'OTHER', 'CARDINAL', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'CARDINAL', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'O

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
from gensim import corpora, models
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [4]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

**1) Prétraitement de données**

In [12]:
def preprocess_data(file_path):
    # lecture du fichier .txt
    
    stop_words = set(stopwords.words('english'))
    
    # Initialisation du lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lire le contenu du fichier
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    processed_data = []
    
    for text in lines:
        # Étape 1: Convertir le texte en minuscules
        text = text.lower()
        # Fin de l'étape 1

        # Étape 2: Suppression des ponctuations
        text = text.translate(str.maketrans("", "", string.punctuation))
        # Fin de l'étape 2

        # Tokenisation
        words = word_tokenize(text)

        # Filtrer les stop words et les mots non alphabétiques
        filtered_words = [word for word in words if word.isalpha() and word not in stop_words]

        # Lemmatisation
        lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

        # Suppression des mots courts (moins de 2 caractères)
        processed_words = [word for word in lemmatized_words if len(word) > 1]

        processed_data.append(processed_words)
    
    # Retourner une liste de listes de mots (un document = une liste de mots)
    return processed_data 

- Execution du prétraitement

**2) Modélisation des sujets**

In [18]:
def topic_modeling(texts, num_topics=3, num_abstracts=None):
    # Sélectionne le nombre d'abstracts à utiliser
    if num_abstracts is not None:
        texts = texts[:num_abstracts]  # Sélectionne les premiers 'num_abstracts' abstracts
    
    # Crée un dictionnaire à partir des textes
    dictionary = corpora.Dictionary(texts)
    
    # Convertit les textes en une représentation de sac de mots (bag-of-words)
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    # Entraîne un modèle LDA (Latent Dirichlet Allocation) sur le corpus
    ldamodel = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    
    # Récupère les topics (thèmes) découverts par le modèle
    topics = ldamodel.print_topics(num_words=4)
    
    return topics


- Execution du topic_modeling

In [ ]:
# //////////////////////////////

**3) Utilisation de CountVectorizer et LDA de sklearn**

In [19]:
# Utilisation de CountVectorizer et LDA de sklearn pour la modélisation des sujets
def sklearn_topic_modeling(texts, num_topics=3, num_abstracts=None):
    # Sélectionne le nombre d'abstracts à utiliser
    if num_abstracts is not None:
        texts = texts[:num_abstracts]  # Sélectionne les premiers 'num_abstracts' abstracts
    
    # Combiner les textes en une seule chaîne pour chaque document
    combined_texts = [" ".join(text) for text in texts]
    
    # Utiliser CountVectorizer pour transformer les textes
    count_vectorizer = CountVectorizer(ngram_range=(1,2), stop_words='english', 
                                       token_pattern="\\b[a-z][a-z]+\\b", lowercase=True, max_features=1000)
    X = count_vectorizer.fit_transform(combined_texts)
    
    # Utiliser LDA de sklearn pour la modélisation des sujets
    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42, learning_method='online')
    lda.fit(X)
    
    return lda, count_vectorizer


**4) Afficher les topics**

In [20]:
def display_topics(model, feature_names, no_top_words, categories=None):
    for ix, topic in enumerate(model.components_):
        if categories:
            print(f"Topic {ix}: {categories[ix]}")
        else:
            print(f"Topic {ix}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

**5) Application de la modélisation des sujets**

In [21]:
def apply_topic_modeling(abstracts, num_abstracts=None):
    
    if num_abstracts is not None:
        abstracts = abstracts[:num_abstracts]
    
    lda, count_vectorizer = sklearn_topic_modeling(abstracts, num_topics=3, num_abstracts=num_abstracts)
    
    display_topics(lda, count_vectorizer.get_feature_names_out(), 20)
    
    return lda


In [23]:
# # Exemple d'utilisation
# # Modélisation des sujets pour un seul résumé
# print("Topics for 1 abstract:")
# single_abstract_topics = apply_topic_modeling(abstracts, num_abstracts=1)

# # Modélisation des sujets pour 10 résumés
# print("\nTopics for 10 abstracts:")
# ten_abstracts_topics = apply_topic_modeling(abstracts, num_abstracts=10)

# # Modélisation des sujets pour tous les résumés
# print("\nTopics for all abstracts:")
# all_abstracts_topics = apply_topic_modeling(abstracts)


In [24]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Chemin du fichier contenant les abstracts
file_path = 'output.txt'

# Prétraitement des données
processed_data = preprocess_data(file_path)

# Application de la modélisation des sujets
apply_topic_modeling(processed_data)

Topic 0:
percent function cee lower rehydration baseline space earth sympathetic study group lung lung function gene greater oral percent percent use mission hour
Topic 1:
patient mutation acute gene protein steroidogenic oxygen regulatory protein acute regulatory regulatory steroidogenic acute human chemical percent affected pressure acute gastroenteritis data loss gastroenteritis
Topic 2:
patient risk cancer group mortality increased significant study treatment increase rr ci puva rr ci activity cell noradrenaline clinical rate digitalis


LatentDirichletAllocation(learning_method='online', n_components=3,
                          random_state=42)